In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
import numpy as np
from collections import Counter
%matplotlib inline 

ImportError: No module named 'tqdm'

In [3]:
df=pd.read_csv("dataset/apache_access.csv",header=None)

In [4]:
df.head()

,0
0,70.69.152.165 - - [Tue Feb 22 19:39:27 UTC 201...
1,147.106.118.104 - - [Tue Feb 22 18:00:37 UTC 2...
2,65.98.119.36 - - [Tue Feb 22 16:32:25 UTC 2017...
3,147.106.118.104 - - [Tue Feb 22 18:18:59 UTC 2...
4,65.98.119.36 - - [Tue Feb 22 16:08:01 UTC 2017...


In [5]:
def grp(patt,txt):
    t=re.search(patt,txt)
    return t.group(0)
df.iloc[:,0]=sorted(df.iloc[:,0],key=lambda l:grp("\[.*?\]",l))

In [58]:
patt=re.compile("(GET|POST)")
v=df.iloc[:,0].apply(lambda x: patt.search(x).group(0))
v.describe()

count     9946
unique       1
top        GET
freq      9946
Name: 0, dtype: object

In [6]:
entries=[i[1] for i in df.itertuples()]
print(entries[100])

14.47.165.89 - - [Tue Feb 22 15:03:47 UTC 2017] "GET /_css/master.1334356838.css HTTP/1.1" 304 6929 "http://www.bing.com/search?q=sumo%20applications&src=IE-SearchBox&FORM=IE11SR" "Mozilla/5.0 (BlackBerry; U; BlackBerry 9900; en) AppleWebKit/534.11+ (KHTML, like Gecko) Version/7.1.0.346 Mobile Safari/534.11+"


In [68]:
ipList=[]
for i in entries:
    ip=re.findall("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",i)[0]
    ipList.append(tuple(map(int,ip.split('.'))))
ipList=list(zip(*ipList))
print(len(ipList)) #4 Parts of IP

4


In [69]:
fileList=[]
for i in entries:
    file=re.search("GET (.*?) HTTP/1.1",i).group(1)
    fileList.append(file)
from sklearn.preprocessing import LabelEncoder
encodedFiles=LabelEncoder().fit_transform(fileList)

In [147]:
d=np.array([ipList[0],ipList[1],ipList[2],ipList[3],encodedFiles],dtype=np.float32).T
print(d.shape)

(9946, 5)


In [162]:
def dist(P1,P2):
    return np.linalg.norm(P1-P2)
def h_avg(l):
    if (l==0).any():
        np.place(l,l==0,0.123456)
    return l.shape[0]/np.sum(1/l,axis=0)
def kmeans(d,k=5,steps=10000):
    centroid=d[np.random.randint(0,d.shape[0]-1,size=k)]
    point2centroid=np.zeros(d.shape[0])
    for step in tqdm(range(steps)):
        for di,i in enumerate(d):
            dist_vec=np.zeros(centroid.shape[0])
            for ci,c in enumerate(centroid):
                dist_vec[ci]=dist(c,i)
            point2centroid[di]=np.argmin(dist_vec)
        for ci,c in enumerate(centroid):
            tp=h_avg(d[[i for i,x in enumerate(point2centroid) if x==ci]])
            centroid[ci,:]=tp
    for i,c in enumerate(centroid):
        am=abs(np.argmin(c))
        if np.int32(c[am])==0:
            centroid[i][am]=0
            
    return centroid,point2centroid

In [163]:
centroid,point2centroid=kmeans(d,3)
print(v)
percent=30
train_c,test_c=centroid[:centroid.shape[0]*(percent/100)],centroid[centroid.shape[0]*(percent/100):]
train_p,test_p=point2centroid[:point2centroid.shape[0]*(percent/100)],centroid[point2centroid.shape[0]*(percent/100):]

  0%|          | 28/10000 [00:12<1:17:08,  2.15it/s]

KeyboardInterrupt: 

In [ ]:
def autoRegressorModel():
    pass